In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
import os
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.utils.data.dataset import Dataset
import numpy as np
import matplotlib.pyplot as plt
import argparse
import math

from glob import glob
from torch.utils.data import Dataset, DataLoader
import os.path
from PIL import Image
from lie_learn.groups.SO3 import change_coordinates as SO3_coordinates
from tensorboardX import SummaryWriter

from lie_vae.vae import VAE
from lie_vae.reparameterize import Nreparameterize, SO3reparameterize, N0reparameterize
from lie_vae.utils import MLP, random_split, View, Flatten
from lie_vae.lie_tools import group_matrix_to_eazyz, block_wigner_matrix_multiply
from lie_vae.datasets import CubeDataset
from lie_vae.nets import CubesDeconvNet, CubesConvNet
from lie_vae.decoders import ActionNet, MLPNet


CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument('--latent', '-z', type=str, default='so3', help='normal or so3')
parser.add_argument('--decoder', '-d', type=str, default='action', help='mlp or action')
parser.add_argument('--epochs', '-e', type=int, default=200, help='')
parser.add_argument('--load', '-l', type=str, default='', help='')
parser.add_argument('--save', '-s', type=str, default='', help='')
parser.add_argument('--batch_dim', '-b', type=int, default=32, help='')

FLAGS, unparsed = parser.parse_known_args()

In [ ]:
class ConvAE(nn.Module):
    def __init__(self):
        super(ConvAE, self).__init__()
        
        ndf = 16
        self.encoder = nn.Sequential(
            CubesConvNet(),
            nn.Linear(ndf * 4, 3)
        )

        deconv = CubesDeconvNet((6+1)**2 * 100, 50)
        if FLAGS.decoder == "mlp":
            if FLAGS.latent == "so3":
                self.decoder = MLPNet(in_dims=9, degrees=6, rep_copies=100, deconv=deconv)
            elif FLAGS.latent == "normal":
                self.decoder = MLPNet(in_dims=3, degrees=6, rep_copies=100, deconv=deconv)
        elif FLAGS.decoder == "action":
            self.decoder = ActionNet(6, rep_copies=100, with_mlp=True, deconv=deconv)
        
    def forward(self,x):
        lie_algebra_el = self.encoder(x)
        
        if FLAGS.decoder == "action":
            if FLAGS.latent == "so3":
                group_el = SO3reparameterize._expmap_rodrigues(lie_algebra_el)
                angles = group_matrix_to_eazyz(group_el)
            elif FLAGS.latent == "normal":
                angles = F.tanh(lie_algebra_el)
                angles = angles * torch.tensor([[math.pi, math.pi / 2, math.pi]], device=device) + \
                    torch.tensor([[0, math.pi / 2, 0]], device=device)

    #         angles = group_matrix_to_quaternions(group_el)
            x_recon = self.decoder(angles)
        
        elif FLAGS.decoder == "mlp":
            if FLAGS.latent == "so3":
                out = SO3reparameterize._expmap_rodrigues(lie_algebra_el)
            elif FLAGS.latent == "normal":
                out = lie_algebra_el
                
            x_recon = self.decoder(out)
        return x_recon
    
    def recon_loss(self, x_recon, x):
        x = x.expand_as(x_recon)
        max_val = (-x_recon).clamp(min=0)
        loss = x_recon - x_recon * x + max_val + ((-max_val).exp() + (-x_recon - max_val).exp()).log() 
        
        return loss.sum(-1).sum(-1).sum(-1)

In [ ]:
train_dataset = CubeDataset('train')
dev_dataset = CubeDataset('dev')
test_dataset = CubeDataset('test')

train_loader = data_utils.DataLoader(train_dataset, batch_size=FLAGS.batch_dim, shuffle=True)
dev_loader = data_utils.DataLoader(dev_dataset, batch_size=FLAGS.batch_dim, shuffle=True)
test_loader = data_utils.DataLoader(test_dataset, batch_size=FLAGS.batch_dim, shuffle=True)

In [ ]:
model = ConvAE().cuda() if CUDA else ConvAE()

if FLAGS.load != '':
    model = torch.load(FLAGS.load)
    
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
print ("training {} with {}".format(FLAGS.latent, FLAGS.decoder))
for j in range(FLAGS.epochs):
    for i, (images, _) in enumerate(train_loader):
        images = Variable(images).cuda() if CUDA else Variable(images)

        optimizer.zero_grad()
        
        images_recon = model(images)
        
#         loss = (images_recon - images).pow(2).sum(-1).sum(-1).sum(-1).mean()
        loss = model.recon_loss(images_recon, images).mean()
        loss.backward()
        optimizer.step()
        print('\r epoch: {:4}/{:4}, it: {:4}/{:4}: loss: {:.4f},'.format(
                j, FLAGS.epochs, i, len(train_loader),
                float(loss.data.cpu().numpy())),
            end='')

    if FLAGS.save != '':
        torch.save(model, FLAGS.save)
        
    loss = np.array([model.recon_loss(model(Variable(images).cuda() if CUDA else Variable(images)), 
                           Variable(images).cuda() if CUDA else Variable(images)).mean().data.cpu().numpy()
         for i, (images, _) in enumerate(dev_loader)]).mean()
    
    print('\r epoch: {:4}/{:4}, it: {:4}/{:4}: loss: {:.4f},'.format(
                j, FLAGS.epochs, i, len(train_loader), loss))

In [ ]:
loss = np.array([model.recon_loss(model(Variable(images).cuda() if CUDA else Variable(images)), 
                           Variable(images).cuda() if CUDA else Variable(images)).mean().data.cpu().numpy()
 for i, (images, _) in enumerate(test_loader)]).mean()
print("TEST LOSS --> {}".format(loss))

In [ ]:
img = Variable(next(iter(train_loader))[0][0:1])
rec_img = model(img.cuda())

plt.imshow(img.data.cpu().numpy()[0].transpose(1, 2, 0))
plt.show()
plt.imshow(F.sigmoid(rec_img).data.cpu().numpy()[0].transpose(1, 2, 0))
plt.show()

## Continuity checks

In [ ]:
# degrees = 3
# matrix_dims = (degrees + 1) ** 2
# in_dims = 2
# item_rep = nn.Parameter(torch.randn((matrix_dims, in_dims)))

In [ ]:
# N = 1000
# x0 = np.random.normal(0,0.001,(N+1,3))
# print(x0.shape)
# x0 = np.cumsum(np.cumsum(x0,-2), -2)
# print(x0.shape)
# x0 = torch.tensor(x0, dtype = torch.float32)
# group_el = SO3reparameterize._expmap_rodrigues(x0)
# angles = group_matrix_to_eazyz(group_el)


# plt.plot(angles.detach().numpy()[:,0], "o", color = "b")
# plt.show()
# plt.plot(angles.detach().numpy()[:,1], "o", color = "b")
# plt.show()
# plt.plot(angles.detach().numpy()[:,2], "o", color = "b")
# plt.show()

In [ ]:

# n = angles.size(0)
# item_expanded = item_rep.expand(n, -1, -1)
# item = block_wigner_matrix_multiply(angles, item_expanded, degrees) \
#             .view(-1, matrix_dims * in_dims)

# i = np.random.randint(item.shape[1])
   
# plt.plot(item.detach().numpy()[:,i], "o", color = "b")
# plt.show()
